In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from time import time
from keras.preprocessing.image import ImageDataGenerator



from keras.datasets import cifar10
import random
import cv2
import numpy as np
from keras.utils import to_categorical

c:\users\jesper\appdata\local\conda\conda\envs\tensorflowv3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Configure the loading of data
# Image Generator
train_datagen = ImageDataGenerator()

val_datagen = ImageDataGenerator()

batch_size = 16
Classes = 12
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../../LocalContent/Datasets/Resized_180/train',  # this is the target directory
       # target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        '../../LocalContent/Datasets/Resized_180/verf',
      #  target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 4255 images belonging to 12 classes.
Found 495 images belonging to 12 classes.


In [3]:
# Configure model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(Classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [4]:
# Fit the model for the top layers
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit_generator(
        train_generator,
        steps_per_epoch=4224/batch_size // batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=480/batch_size // batch_size)

Epoch 1/1
3/4 [=====================>........] - ETA: 3s - loss: 7.8995

UnboundLocalError: local variable 'outs' referenced before assignment

In [ ]:
# Fit the model for the lower layers
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

tbCallBack = keras.callbacks.TensorBoard(log_dir="../../LocalContent/Logs/KerasRetrainInceptionv3_3/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)


# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(x_train_resized, y_train_hot_encoded, epochs=5,callbacks=[tbCallBack])